In [ ]:
!pip install mediapipe
!pip install deepface

In [ ]:
import numpy as np
import cv2
from google.colab.patches import cv2_imshow
# from mediapipe.framework.formats import landmark_pb2
# face_landmarks_proto = landmark_pb2.NormalizedLandmarkList()
# face_landmarks_proto.landmark.extend([
#       landmark_pb2.NormalizedLandmark(x=landmark.x, y=landmark.y, z=landmark.z) for landmark in face_landmarks
#     ])
def detect_eyes(image,face):
  # image.flags.writeable = False
  # image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # frame to RGB for the face-mesh model
  # results = face_mesh.process(image)
  # # image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
  # if results.multi_face_landmarks==None:
  #   return None
  # 316, 357
  # 31, 374
  relative = lambda landmark, shape: (int(landmark.x * shape[1]), int(landmark.y * shape[0]))
  image_points = np.array([
          relative(face.landmark[336], image.shape),
          relative(face.landmark[340], image.shape),
          relative(face.landmark[21],image.shape),
          relative(face.landmark[233],image.shape)
      ])
  # cv2.rectangle(image,image_points[0],image_points[1],(0,0,255),2)
  return image_points
  # cv2_imshow(image)

In [ ]:
# import mediapipe as mp
# from deepface import DeepFace

# mp_face_mesh = mp.solutions.face_mesh # initialize the face mesh model

# # camera stream:
# cap = cv2.VideoCapture("/content/10sec_no_sound.mp4")
# output_video_path = '/content/output_video_deepface.mp4'
# fps = int(cap.get(cv2.CAP_PROP_FPS))
# frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
# frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
# fourcc = cv2.VideoWriter_fourcc(*'mp4v')
# out = cv2.VideoWriter(output_video_path, fourcc, fps, (frame_width, frame_height))
# print(cap.isOpened())
# with mp_face_mesh.FaceMesh(
#         max_num_faces=3,                            # number of faces to track in each frame
#         refine_landmarks=True,                      # includes iris landmarks in the face mesh model
#         min_detection_confidence=0.1,
#         min_tracking_confidence=0.1) as face_mesh:

#     while cap.isOpened():


#         success, image = cap.read()

#         if not success:                            # no frame input
#             break
#         # To improve performance, optionally mark the image as not writeable to
#         # pass by reference.
#         image.flags.writeable = False
#         image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # frame to RGB for the face-mesh model
#         results = face_mesh.process(image)
#         image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

#         if results.multi_face_landmarks:
#             image=detect_eyes(image, results.multi_face_landmarks)
#         # print(results.multi_face_landmarks[0])
#         # cv2.imshow('output window', image)
#         out.write(image)
#         if cv2.waitKey(2) & 0xFF == 27:
#             break
# cap.release()
# out.release()

In [ ]:
# print(len(results.multi_face_landmarks))
# print(results.multi_face_landmarks[0].landmark[336])
# results.multi_face_landmarks[0].landmark[340]

In [ ]:
import mediapipe as mp
from deepface import DeepFace
import cv2

mp_face_mesh = mp.solutions.face_mesh # initialize the face mesh model
face_mesh=mp_face_mesh.FaceMesh(max_num_faces=1,refine_landmarks=True,min_detection_confidence=0.3,min_tracking_confidence=0.3)
cap = cv2.VideoCapture("/content/10sec_no_sound.mp4")
output_video_path = '/content/output_test_deepface.mp4'
fps = int(cap.get(cv2.CAP_PROP_FPS))
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_video_path, fourcc, fps, (frame_width, frame_height))
while cap.isOpened():
  ret, frame = cap.read()
  if not ret:
      break

  detected_faces = DeepFace.extract_faces(frame, detector_backend='retinaface')

  for face in detected_faces:
    x, y, w, h = int(face['facial_area']['x']), int(face['facial_area']['y']), int(face['facial_area']['w']), int(face['facial_area']['h'])
    # print(x,y,w,h)
    # print(frame[x])
    image=frame[y:y+h,x:x+w,:]
    image.flags.writeable = False
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # frame to RGB for the face-mesh model
    results = face_mesh.process(image)
    # image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    # eye_points=detect_eyes(frame[y:y+h,x:x+w,:],face_mesh)
    if results.multi_face_landmarks:
      eye_points=detect_eyes(image, results.multi_face_landmarks[0])
    # if eye_points!=None:
      cv2.rectangle(frame, (x+eye_points[0][0],y+eye_points[0][1]), (x+eye_points[1][0],y+eye_points[1][1]), (0, 255, 0), 3)
      cv2.rectangle(frame, (x+eye_points[2][0],y+eye_points[2][1]), (x+eye_points[3][0],y+eye_points[3][1]), (0, 255, 0), 3)

  out.write(frame)

cap.release()
out.release()


In [ ]:
# frame[y:y+h,x:x+w,:].shape

In [ ]:
# !pip install batch-face
# import cv2
# from batch_face import RetinaFace

# detector = RetinaFace(gpu_id=0)
# img = cv2.imread("examples/obama.jpg")
# faces = detector(img, cv=True) # set cv to False for rgb input, the default value of cv is False
# box, landmarks, score = faces[0]